## Desafío 2 - Procesamiento del Lenguaje Natural I
##### Docentes: Rodrigo Cárdenas / Nicolás  Vattuone
##### Autora: María Luz Micozzi

- Crear sus propios vectores con Gensim
basado en lo visto en clase con otro
dataset.
- Probar términos de interés y explicar
similitudes en el espacio de embeddings.
- Intentar plantear y probar tests de
analogías.
- Graficar los embeddings
resultantes.
- Sacar conclusiones.

Se utilizará para este desafío el libro ***“Orgullo y Prejuicio”*** en español. El mismo fue descargado del sitio web [textos.info](https://www.textos.info/jane-austen/orgullo-y-prejuicio/descargar-pdf) y luego subido a [drive](https://drive.google.com/file/d/1oNRvQ4wl8CNbVdVkKyjZQuVm8kInl7Xg/view?usp=sharing) en formato txt.

In [1]:
# !pip install --upgrade numpy==1.24.0
# !pip install --upgrade --force-reinstall gensim

In [2]:
# imports
import os
import gdown
import pandas as pd
import re
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.manifold import TSNE
import numpy as np
from nltk.corpus import stopwords
import nltk
from collections import Counter
import plotly.express as px

In [3]:
# Descargamos el libro desde drive
# Link = https://drive.google.com/file/d/1oNRvQ4wl8CNbVdVkKyjZQuVm8kInl7Xg/view?usp=drive_link

file_id = '1oNRvQ4wl8CNbVdVkKyjZQuVm8kInl7Xg'
output = 'orgullo_y_prejuicio.txt'

if not os.path.exists(output):
    print("Descargando el archivo desde Google Drive...")
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, output, quiet=False)
else:
    print("El archivo ya está descargado.")

El archivo ya está descargado.


In [4]:
# Armamos el dataset utilizando salto de línea para separar las oraciones/docs
# Limpiamos titulo y autora, líneas en blanco, nro de capítulo y nro de página

def clean_lines(archivo_txt):
    with open(archivo_txt, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Saltar las dos primeras líneas (título y autora)
    lines = lines[2:]

    # Regex para detectar líneas como "Capítulo I", "Capítulo IX", "CAPÍTULO XXV"
    pattern_chapter = re.compile(r'^cap[ií]tulo\s+[ivxlcdm]+$', re.IGNORECASE)

    # Filtrar y limpiar líneas
    clean_lines = []
    for line in lines:
      line = line.strip()
      if not line or line.isdigit() or pattern_chapter.match(line):
          continue

      # Limpieza de guiones
      line = re.sub(r'\b-(\w+)', r'\1', line)
      line = re.sub(r'(\w+)-\b', r'\1', line)
      line = line.replace('—', '')

      clean_lines.append(line)

    # Crear DataFrame
    df = pd.DataFrame(clean_lines, columns=['line'])
    return df


df = clean_lines("orgullo_y_prejuicio.txt")

In [5]:
# Imprimimos las primeras líneas
for i in range(5):
    print(f"Línea {i}: {df['line'][i]}")

Línea 0: Es una verdad mundialmente reconocida que un hombre soltero, poseedor
Línea 1: de una gran fortuna, necesita una esposa.
Línea 2: Sin embargo, poco se sabe de los sentimientos u opiniones de un hombre
Línea 3: de tales condiciones cuando entra a formar parte de un vecindario. Esta
Línea 4: verdad está tan arraigada en las mentes de algunas de las familias que lo


In [6]:
# Imprimir algunas líneas aleatorias
muestras = df.sample(n=5, random_state=42)

for idx, linea in muestras.iterrows():
    print(f"Línea {idx}: {linea['line']}")

Línea 6784: dirigir la mirada a los objetos que le señalaban, no distinguía ninguna parte
Línea 9368: que se abrió la puerta y entró la visita. Era lady Catherine de Bourgh.
Línea 5239: actual relativa pobreza. Usted le negó el porvenir que, como bien debe
Línea 4860: Tengo entendido que el señor Bingley no piensa volver a Netherfield.
Línea 2932: adelante. Collins continuó:


In [7]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 10403


### Tokenización y Vectorización

In [8]:
nltk.download('stopwords') # Descargar las listas de stopwords
spanish_stopwords = set(stopwords.words('spanish')) # Obtener el conjunto de stopwords en español

def tokenize_line(line):
    # Convertir la línea en una lista de palabras (tokens),
    # eliminando puntuación y símbolos definidos en `filters`
    tokens = text_to_word_sequence(
        line,
        filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n¡¿'
    )
    return [t for t in tokens if t not in spanish_stopwords]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
# Tokenizar cada línea en palabras
sentence_tokens = [tokenize_line(line) for line in df['line']]

# Crearmos el modelo generador de vectores
# modelo Skipgram
w2v_model = Word2Vec(min_count=1,
                     window=3,
                     vector_size=50,
                     negative=20,
                     workers=1,
                     sg=1)

# Construir vocabulario
w2v_model.build_vocab(sentence_tokens)

In [10]:
# Palabras más frecuentes

all_words = [word for sentence in sentence_tokens for word in sentence]
top_words = Counter(all_words).most_common(20)

print("Top 20 palabras más frecuentes:")
for palabra, freq in top_words:
    print(f"{palabra}: {freq}")

Top 20 palabras más frecuentes:
elizabeth: 874
darcy: 603
señor: 492
si: 480
tan: 473
bingley: 399
señora: 370
jane: 337
bennet: 336
dijo: 318
usted: 284
wickham: 260
ser: 255
señorita: 254
collins: 230
casa: 221
aunque: 204
dos: 198
hermana: 198
bien: 197


In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 10403
Cantidad de words distintas en el corpus: 10292


### Entrenamiento

In [12]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [13]:
# Entrenamos el modelo generador de vectores
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1397766.875
Loss after epoch 1: 695744.375
Loss after epoch 2: 506255.25
Loss after epoch 3: 467570.75
Loss after epoch 4: 447949.5
Loss after epoch 5: 435489.0
Loss after epoch 6: 417991.25
Loss after epoch 7: 401951.0
Loss after epoch 8: 398862.5
Loss after epoch 9: 393778.5
Loss after epoch 10: 392093.5
Loss after epoch 11: 383913.0
Loss after epoch 12: 379567.0
Loss after epoch 13: 373229.5
Loss after epoch 14: 364925.5
Loss after epoch 15: 358811.0
Loss after epoch 16: 352606.0
Loss after epoch 17: 340987.5
Loss after epoch 18: 323450.0
Loss after epoch 19: 317781.0
Loss after epoch 20: 311589.0
Loss after epoch 21: 303876.0
Loss after epoch 22: 298406.0
Loss after epoch 23: 291872.0
Loss after epoch 24: 286732.0
Loss after epoch 25: 280743.0
Loss after epoch 26: 274962.0
Loss after epoch 27: 271280.0
Loss after epoch 28: 266013.0
Loss after epoch 29: 262725.0
Loss after epoch 30: 258685.0
Loss after epoch 31: 254612.0
Loss after epoch 32: 249746.0
Loss after e

(5214705, 5551500)

### Términos de interés
Se buscará los términos más similares a algunas de las palabras más frecuentes y relevantes de la obra.

In [14]:
key_terms = ["señora", "matrimonio", "elizabeth", "familia", "hermana"]

for term in key_terms:
    if term in w2v_model.wv:
        similar = w2v_model.wv.most_similar(term, topn=5)
        print(f"\nPalabras similares a '{term}':")
        for word, sim in similar:
            print(f"  {word}: {sim:.3f}")
    else:
        print(f"\n'{term}' no encontrada.")


Palabras similares a 'señora':
  bennet: 0.885
  embargó: 0.742
  chispas: 0.720
  dependes: 0.715
  insisto: 0.697

Palabras similares a 'matrimonio':
  perfidia: 0.679
  notificase: 0.650
  contraiga: 0.638
  rigida: 0.612
  ansía: 0.611

Palabras similares a 'elizabeth':
  ruborizada: 0.773
  afirmarle: 0.768
  empaque: 0.764
  elegantemente: 0.762
  darcy: 0.761

Palabras similares a 'familia':
  supiesen: 0.613
  emparentado: 0.612
  esperándolos: 0.597
  deseaba: 0.596
  verían: 0.577

Palabras similares a 'hermana':
  decidimos: 0.653
  implicaban: 0.626
  entristecían: 0.596
  inclinado: 0.588
  reanimase: 0.574



- **Palabras similares a "señora":**  
   Las palabras más similares son *bennet* (apellido clave de la familia principal) y verbos en formas conjugadas o participios (*embargó*, *chispas*, *dependes*, *insisto*). Esto sugiere que la palabra "señora" está muy asociada a personajes y acciones en el contexto narrativo, reflejando que el modelo capta relaciones de personajes y acciones frecuentes junto con términos formales de tratamiento.

- **Palabras similares a "matrimonio":**  
   Los términos asociados son más abstractos y formales: *perfidia* (traición), *notificase* (verbo en subjuntivo), *contraiga* (verbo legal o formal de "contraer matrimonio"), *rígida*, *ansía*. Esto indica que el modelo asocia "matrimonio" con conceptos legales, emocionales y formales presentes en el texto.

- **Palabras similares a "elizabeth":**  
   Se destacan términos descriptivos y emocionales como *ruborizada*, *afirmarle*, *elegantemente*, además de la referencia directa a *darcy*. Esto muestra que el modelo reconoce a Elizabeth como un personaje central y la conecta con acciones y estados emocionales que la rodean.

- **Palabras similares a "familia":**  
   Palabras relacionadas con relaciones familiares y estados, como *supiesen*, *emparentado*, *esperándolos*, *deseaba*, *verían*, sugieren que el modelo entiende "familia" en su sentido social y afectivo, enlazándola con vínculos y expectativas.

- **Palabras similares a "hermana":**  
   Aquí aparecen verbos y estados emocionales como *decidimos*, *implicaban*, *entristecían*, *inclinado*, *reanimase*. Esto indica que el modelo relaciona "hermana" con decisiones, emociones y dinámicas internas, típicas del contexto familiar y personal en la novela.

El modelo Word2Vec entrenado sobre *Orgullo y Prejuicio* refleja las relaciones temáticas y emocionales del texto, capturando asociaciones entre personajes, acciones, estados emocionales y conceptos clave de la obra, aunque algunas palabras similares pueden parecer poco intuitivas, probablemente por el contexto lingüístico o frecuencia de uso en el corpus.

### Tests de analogías

In [15]:
# Función auxiliar para probar analogías
def test_analogy(w2v_model, positive, negative, description):
    try:
        # Verificar si las palabras están en el vocabulario
        for word in positive + negative:
            if word not in w2v_model.wv:
                raise KeyError(f"{word}")

        results = w2v_model.wv.most_similar(positive=positive, negative=negative, topn=3)
        print(f"Analogía: {description} =")
        for word, similarity in results:
            print(f"  {word} (Similitud: {similarity:.4f})")
    except KeyError as e:
        print(f"Palabra no encontrada: {e}")
    print("-" * 50)

# Lista de analogías a probar
analogies = [
    (['longbourn', 'darcy'], ['elizabeth'], 'longbourn - elizabeth + darcy'),
    (['jane', 'darcy'], ['elizabeth'], 'jane - elizabeth + darcy'),
    (['señora', 'hombre'], ['mujer'], 'señora - mujer + hombre'),
    (['esposo', 'mujer'], ['hombre'], 'esposo - hombre + mujer')
]

# Ejecutar las analogías
for positive, negative, description in analogies:
    test_analogy(w2v_model, positive, negative, description)

Analogía: longbourn - elizabeth + darcy =
  numeroso (Similitud: 0.6008)
  ejerciese (Similitud: 0.5695)
  newcastle (Similitud: 0.5604)
--------------------------------------------------
Analogía: jane - elizabeth + darcy =
  bingley (Similitud: 0.8185)
  señor (Similitud: 0.6735)
  influyen (Similitud: 0.6471)
--------------------------------------------------
Analogía: señora - mujer + hombre =
  angustia (Similitud: 0.5947)
  m (Similitud: 0.5328)
  bennet (Similitud: 0.5281)
--------------------------------------------------
Analogía: esposo - hombre + mujer =
  indirecta (Similitud: 0.6026)
  molestara (Similitud: 0.5587)
  revela (Similitud: 0.5509)
--------------------------------------------------


- **longbourn - elizabeth + darcy**
  - Palabras resultantes: *numeroso*, *ejerciese*, *newcastle*
  - Interpretación: La analogía no produce términos directamente relacionados con los personajes ni con ubicaciones clave, salvo posiblemente *newcastle*. El modelo puede estar confundido por la baja frecuencia de ciertas combinaciones, reflejando que el contexto semántico entre estas tres palabras es débil en el corpus reducido.

- **jane - elizabeth + darcy**
  - Palabras resultantes: *bingley*, *señor*, *influyen*
  - Interpretación: Esta analogía es bastante coherente. *Bingley* es la pareja de Jane, así como *Darcy* lo es de Elizabeth, lo que sugiere que el modelo ha captado correctamente esta relación paralela. El resto de palabras reflejan interacciones o dinámicas típicas en el entorno social de la novela.

- **señora - mujer + hombre**
  - Palabras resultantes: *angustia*, *m*, *bennet*
  - Interpretación: Aunque el resultado incluye *bennet*, que podría ser coherente (por *señora Bennet*), las demás palabras no representan una conversión clara de género. El modelo parece captar el entorno emocional o social de la palabra “señora”, pero no logra ajustar bien la dimensión de género con la operación semántica esperada.

- **esposo - hombre + mujer**
  - Palabras resultantes: *indirecta*, *molestara*, *revela*
  - Interpretación: La analogía no arroja palabras femeninas ni relacionadas claramente con el rol de “esposa”. Esto sugiere que el modelo no ha captado con fuerza la estructura de género entre “esposo”, “hombre” y “mujer”, posiblemente por falta de ejemplos suficientes o balanceados en el corpus.

Las analogías que involucran relaciones entre personajes (*Jane*, *Elizabeth*, *Darcy*) son las que arrojan resultados más coherentes, mientras que aquellas que intentan capturar relaciones de género o roles sociales (*señora*, *esposo*) muestran más ruido semántico.


### Gráficos

In [16]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [18]:
# Graficar los embedddings en 2D

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=100
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab")

In [20]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Conclusiones

El modelo de word embeddings entrenado con Word2Vec sobre el texto de Orgullo y Prejuicio en español capturó adecuadamente las relaciones contextuales y temáticas propias de la obra. Las palabras similares reflejan asociaciones relevantes con personajes, acciones y estados emocionales presentes en la narrativa.

Sin embargo, los tests de analogías mostraron resultados mixtos. Las analogías relacionadas con relaciones entre personajes principales obtuvieron respuestas más coherentes, mientras que aquellas que intentaban representar relaciones de género o roles sociales no lograron resultados claros ni consistentes. Esto indica que, aunque el modelo aprende asociaciones locales y específicas del corpus, tiene dificultades para generalizar relaciones semánticas más abstractas o universales.

En conclusión, entrenar embeddings en un corpus literario pequeño y especializado es útil para captar el contexto específico, pero limita la capacidad del modelo para tareas que requieren generalización semántica amplia. Para superar esta limitación, sería recomendable utilizar modelos preentrenados en grandes corpus o combinar embeddings específicos con representaciones más generales.
